In [1]:
import time
import pickle

import numpy as np
import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

In [5]:
# Data Loading
data_dir = './_data/'
data_filenames = ['2022-04-22 02:07:18_0_cont.csv']

df_l = []
for f in data_filenames:
    filepath = data_dir + f
    df_l.append(pd.read_csv(filepath, index_col=None, header=0))

df = pd.concat(df_l, axis=0, ignore_index=True)
df.head()

,paddle_position,paddle_state,previous_paddle_position,ball_position,ball_state,previous_ball_position,keypress,previous_keypress
0,[ 0 56],right,"[0, 0]","[0, 0]",stagnant,"[0, 0]",0,0
1,[ 0 56],stagnant,[ 0 56],"[0, 0]",stagnant,"[0, 0]",0,0
2,[ 0 56],stagnant,[ 0 56],"[0, 0]",stagnant,"[0, 0]",0,0
3,[ 0 56],stagnant,[ 0 56],"[0, 0]",stagnant,"[0, 0]",0,0
4,[ 0 56],stagnant,[ 0 56],"[0, 0]",stagnant,"[0, 0]",0,0


In [8]:
# Construct PGM
model_struct = BayesianNetwork([('ball_state','keypress'), ('paddle_state', 'keypress')])


In [9]:
# Training using MLE
mle = MaximumLikelihoodEstimator(model=model_struct, data=df)

In [11]:
# Print Learned Parameters
print(mle.estimate_cpd(node="keypress"))

+--------------+--------------------+-----+------------------------+
| ball_state   | ball_state(left)   | ... | ball_state(stagnant)   |
+--------------+--------------------+-----+------------------------+
| paddle_state | paddle_state(left) | ... | paddle_state(stagnant) |
+--------------+--------------------+-----+------------------------+
| keypress(0)  | 0.0                | ... | 0.9394812680115274     |
+--------------+--------------------+-----+------------------------+
| keypress(1)  | 0.0                | ... | 0.040345821325648415   |
+--------------+--------------------+-----+------------------------+
| keypress(2)  | 0.0                | ... | 0.01440922190201729    |
+--------------+--------------------+-----+------------------------+
| keypress(3)  | 1.0                | ... | 0.005763688760806916   |
+--------------+--------------------+-----+------------------------+
